<a href="https://colab.research.google.com/github/alonsocampana/fire-montesinho/blob/main/Hyperparameter_tuning_classificators_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#pip install scikit-optimize

     |████████████████████████████████| 101 kB 9.3 MB/s 


In [134]:
import sys
sys.path.insert(1, './imports')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from exploratory_analysis import *
from preprocessing import *
from model_selection import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import skopt
pd.options.display.max_rows = 30
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt import forest_minimize

# Searching the model space for most accurate (jan-may)

In [117]:
search_space = list()
search_space.append(Real(1e-6, 100.0, 'log-uniform', name='C'))
search_space.append(Categorical(['rbf'], name='kernel'))
search_space.append(Integer(1, 5, name='degree'))
search_space.append(Real(1e-6, 100.0, 'log-uniform', name='gamma'))
@use_named_args(search_space)
def evaluate_model(**params):
	# configure the model with specific hyperparameters
	model = SVC()
	model.set_params(**params)
	# define test harness
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=3558)
	# calculate 5-fold cross validation
	result = cross_val_score(model, X, y, cv=cv, n_jobs=-1, scoring='accuracy')
	# calculate the mean of the scores
	estimate = np.mean(result)
	# convert from a maximizing score to a minimizing score
	return 1.0 - estimate

In [155]:
fires = pd.read_csv("fires_jan_may.csv")

In [156]:
X, y = fires.drop(["area", "area_bool", "index", "index.1", "index.2", 'Unnamed: 0', 'index'], axis=1), fires["area_bool"]

In [120]:
result = gp_minimize(evaluate_model, search_space)

/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packa

In [121]:
print(result.x)
print(result.fun)

[100.0, 'rbf', 5, 0.002652859881404326]
0.29055555555555557


# Searching Regression with smaller loss

## Lasso

In [157]:
filter_nonzero = fires["area_bool"] == 1
X, y = fires[filter_nonzero].drop(["area", "area_bool", "index", "index.1", "index.2", 'Unnamed: 0', 'index'], axis=1), fires[filter_nonzero]["area"]

In [128]:
losses_lasso_df = hyperpar_grid_lasso(X, y, degrees = [1, 2, 3, 4])

In [129]:
losses_lasso_df

,0.100000,0.143845,0.206914,0.297635,0.428133,0.615848,0.885867,1.274275,1.832981,2.636651,3.792690,5.455595,7.847600,11.288379,16.237767,23.357215,33.598183,48.329302,69.519280,100.000000
1,280.532749,278.108662,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243
2,280.532749,278.108662,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243
3,280.532749,278.108662,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243
4,280.532749,278.108662,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243,277.926243


## Ridge

In [163]:
def crossval_ridge(X, y, degree, alpha):
    """
        Uses k-fold crossval to estimate the loss associated to a given degree of a polynomial feature map, and alpha regularization value.
    """
    X_temp = X.copy()
    featurizer = PolynomialFeatures(degree=degree)
    featurizer.fit(X)
    X_temp = featurizer.transform(X_temp)
    rskf = KFold(n_splits=5, shuffle=True,random_state=3558)
    losses = []
    n=0
    for train_index, test_index in rskf.split(X, y):
        X_train, X_test = X_temp[train_index], X_temp[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model = Ridge(alpha=alpha)
        n+=1
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        losses.append(average_squared_loss_from_log(y_pred, y_test))
    return sum(losses)/n

def hyperpar_grid_ridge(X, y, degrees = [1, 2, 3], min_alpha = -1, max_alpha = 2):
    """
        Uses grid search for estimating the loss associated to different combinations of degrees of the feature map and alpha regularization parameters
    """
    degrees = degrees
    alphas = np.logspace(min_alpha, max_alpha, 20)
    losses = np.zeros([len(degrees), len(alphas)])
    for i, deg in enumerate(degrees):
        for j, alpha in enumerate(alphas):
            losses[i, j] = crossval_ridge(X, y, deg, alpha)
    return pd.DataFrame(losses, index= degrees, columns = alphas)
losses_ridge_df = hyperpar_grid_ridge(X, y, degrees = [1, 2, 3, 4])

In [164]:
losses_ridge_df

,0.100000,0.143845,0.206914,0.297635,0.428133,0.615848,0.885867,1.274275,1.832981,2.636651,3.792690,5.455595,7.847600,11.288379,16.237767,23.357215,33.598183,48.329302,69.519280,100.000000
1,388.470804,320.782362,279.571537,256.553774,246.074190,243.858677,246.481037,251.286181,256.488693,261.150673,264.960824,267.946515,270.256650,272.049249,273.450901,274.551311,275.411837,276.076932,276.582586,276.960276
2,300.646985,295.885875,290.318361,284.214510,278.019636,272.281350,267.529248,264.158257,262.351762,262.048662,262.957732,264.637804,266.634457,268.605841,270.369173,271.868846,273.117626,274.150806,275.002338,275.697703
3,281.708523,281.211956,280.532915,279.623202,278.438100,276.951131,275.175381,273.186299,271.134916,269.238423,267.741554,266.854679,266.686652,267.200680,268.223895,269.516716,270.862365,272.121267,273.232425,274.185145
4,274.999590,274.899425,274.758012,274.559954,274.285666,273.911752,273.413048,272.767552,271.965307,271.021133,269.988411,268.967631,268.101407,267.549417,267.443389,267.835155,268.666347,269.787318,271.019644,272.219692


## Gradient boost regressor with polynomial features

In [170]:
from sklearn.ensemble import GradientBoostingRegressor
def gbr_score(X, y, degree, learning_rate, n_estimators):
  poly = PolynomialFeatures(degree=degree)
  poly.fit(X)
  X_temp = poly.transform(X)
  rskf = KFold(n_splits=5, shuffle=True,random_state=3558)
  losses = []
  n=0
  for train_index, test_index in rskf.split(X, y):
    n += 1
    X_train, X_test = X_temp[train_index], X_temp[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = GradientBoostingRegressor(learning_rate = learning_rate, n_estimators = n_estimators)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    losses.append(average_squared_loss_from_log(y_pred, y_test))
  return sum(losses)/n
gbr_score(X, y, 2, 0.1, 200)

def hyper_opt_gbr(X, y):
  degrees, learning_rates, n_estimatorss = [1, 2, 3], np.linspace(0.05, 0.6, 5), [20, 50, 100]
  min_loss = 10000
  for deg in degrees:
    for lr in learning_rates:
      for n_estimator in n_estimatorss:
        temp_loss = gbr_score(X, y, deg, lr, n_estimator)
        if temp_loss < min_loss:
          min_loss = temp_loss
          min_deg = deg
          min_lr = lr
          min_n_estimator = n_estimator
  return {"loss":min_loss, "deg": min_deg, "lr":min_lr, "n_estimators":min_n_estimator}

hyper_opt_gbr(X, y)

{'deg': 3, 'loss': 227.0846601173799, 'lr': 0.1875, 'n_estimators': 20}